# Regularization in Machine Learning Models

**Yuanzhe(Roger) Li, 2020-01-07**

In [7]:
import numpy as np
from sklearn import linear_model

Outline
- Regularized Regression
    - Ridge Regression
    - LASSO
- (Tentative header) Structured Sparsity

# Regularized Regression
## Linear regression recap

Recall that in the context of linear regression $\mathbf{y} = \mathbf{X \beta} + \mathbf{\epsilon}$ where $\epsilon_i \overset{\text{iid}}{\sim} \mathcal{N}(0,\sigma^2)$.

Say $\mathbf{y} \in \mathbb{R}^n$, the negative log-likelihood is 
$$L(\mathbf{\beta}|\mathbf{X, y}) = \dfrac{n}{2}\log (2\pi \sigma^2) + \dfrac{1}{2\sigma^2}\sum_{i=1}^n(y_i -\mathbf{x}_i^T\mathbf{\beta})^2$$
Notice that the **MLE** is also the least squares soltion 
$$\mathbf{\beta}^* = \mathbf{\beta}_{MLE} = \mathbf{\beta}_{LS} = \underset{\mathbf{\beta}}{\text{arg min}}\sum_{i=1}^n(y_i -\mathbf{x}_i^T\mathbf{\beta})^2$$ 

## Linear regression recap (continued)
In matrix form 
$$\begin{align}
\mathbf{\beta}^* & = \underset{\mathbf{\beta}}{\text{arg min}} [(\mathbf{y - X\beta})^T(\mathbf{y - X\beta})] \\ 
& = \underset{\mathbf{\beta}}{\text{arg min}}[\mathbf{y}^T\mathbf{y} -2\mathbf{\beta}^T\mathbf{X}^T\mathbf{y} + \mathbf{\beta}^T \mathbf{X}^T\mathbf{X \beta}]  \end{align}$$
Denote $\mathbf{e} = \mathbf{y - X\beta} $, and set 

$$\dfrac{\partial \mathbf{e}^T \mathbf{e}}{\partial \mathbf{\beta}} = -2 \mathbf{X}^T\mathbf{y} + 2 \mathbf{X}^T\mathbf{X}\mathbf{\beta} = \mathbf{0}$$

we have 
$$
\begin{align}
& \mathbf{X}^T\mathbf{X}\mathbf{\beta} = \mathbf{X}^T\mathbf{y} \\ 
\Rightarrow \quad & \mathbf{\beta}^* = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T \mathbf{y}
\end{align}
$$

## Penalized loss function
The loss function of regularized regression takes the following form:

$$Loss(\mathbf{\beta}) = \mathbf{E}(\mathbf{\beta}) + \frac{\lambda}{2} \cdot \mathbf{P}(\mathbf{\beta})$$
where 

- $\mathbf{E}(\mathbf{\beta}) = \dfrac{1}{2}\sum_{i=1}^n(y_i -\mathbf{x}_i^T\mathbf{\beta})^2$ is the sum-of-squares error,
- $\mathbf{P}$ is a penalty function that aims at regularizing the unknown parameters, and 
- $\lambda$ is a hyperparameter that controls the tradeoff between the two components of the loss function.

## Ridge regression

### Penalty
One choice of $P_\lambda$ that favors small regression coefficients is to penalize the sum of squares of the regression coefficients:

$$P (\mathbf{\beta}) = \sum_{j=1}^p\beta_j^2$$

and the whole loss function becomes

$$Loss(\mathbf{\beta}) = \frac{1}{2}\sum_{i=1}^n(y_i -\mathbf{x}_i^T\mathbf{\beta})^2 + \frac{\lambda}{2}\sum_{j=1}^p\beta_j^2$$

which is known as ***ridge regression*** in statistics.

## Ridge regression

### Solution 

Since the loss function of ridge regression is differentiable, it has a simple closed-form solution

$$\hat{\mathbf{\beta}} = (\mathbf{X}^T\mathbf{X} + \lambda \mathbf{I})^{-1}\mathbf{X}^T \mathbf{y}$$

The solution is similar to the least squares solution with an extra diagonal matrix (i.e., $\lambda \mathbf{I}$) to be inverted, which improves the numeric stability of the system when $\mathbf{X}^T\mathbf{X}$ is close to being non-invertible 

*Question: what does that mean in practice?*




## Ridge regression

### A simple example

In [9]:
# Generate data
np.random.seed(2)
x1 = np.random.normal(size=20)
x2 = np.random.normal(loc = x1, scale=0.01, size=20)
y = np.random.normal(loc = 3 + x1 + x2, size=20)

# Train an OLS model
design_mat = np.hstack((np.array(x1).reshape(20,1), np.array(x2).reshape(20,1)))
reg_ols = linear_model.LinearRegression()
reg_ols.fit(design_mat, y)
reg_ols.coef_

array([ 14.51690508, -12.25821163])

In [21]:
# Use ridge penalty
reg_ridge = linear_model.Ridge(alpha = 1.0)
reg_ridge.fit(design_mat, y)
reg_ridge.coef_

array([1.10585803, 1.08303544])

In [4]:
from sklearn.linear_model import TheilSenRegressor
from sklearn.datasets import make_regression
X, y = make_regression(n_samples=200, n_features=3, noise=4.0, random_state=0)
reg = TheilSenRegressor(random_state=0).fit(X, y)
reg.score(X, y)

0.9979467923453923

- Huber regression is a robust regression method that uses the [Huber loss](https://en.wikipedia.org/wiki/Huber_loss), which applies a linear loss (as opposed to quadratic loss) to samples that are classified as outliers (i.e., absolute error $> \epsilon$)
- In scikit-learn, the [`HuberRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.HuberRegressor.html#sklearn.linear_model.HuberRegressor) minimizes the following loss function:

$$\min_{\omega, \sigma} \sum_{i=1}^n(\sigma + H_\epsilon(\frac{X_i\omega - y_i}{\sigma})\sigma) + \alpha ||\omega||_2^2$$

where $H_\epsilon(z) = z^2 \text{ if } |z| < \epsilon$, and $H_\epsilon(z) = \epsilon|z| - \epsilon^2 \text{ otherwise}$.